## 5a
We collect the GDP growth data of Vietnam from 1985 to 2023
The factors that affect economic growth in Vietnam are:
- FDI inflows & outflows
- Contribution of the service sector since in the most recent year, Vietnam’s service sector contributed the largest percentage to the country’s gross domestic product (GDP).
- Trade & Investment (?)


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [11]:
df = pd.read_csv("data.csv")
df

,Unnamed: 0,1985,1986,1987,1988,1989,1990,1991,1992,1993,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,GDP growth rate (%),3.805856,2.789292,3.58347,5.135012,7.364513,5.100918,5.960844,8.646047,8.072731,...,6.422243,6.987154,6.690009,6.94019,7.465007,7.359263,2.865413,2.553729,8.123514,5.046431


In [ ]:
plt.plot()